In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import shutil
import torch.distributed as dist
import math
from PIL import Image
from tqdm import tqdm
from glob import glob
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from typing import Type, Any, Callable, Union, List, Optional
from torch import Tensor
from torch.hub import load_state_dict_from_url
from torch.utils.data.dataloader import default_collate
from torch.utils.data.sampler import BatchSampler
from collections.abc import Mapping, Sequence

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"


In [2]:
train_path_path= f"../data/split_train/train"
test_path_path= f"../data/split_train/val"

In [3]:
batch_size=256
num_gpus=4
num_workers=8
lr=0.001

In [4]:
normalize = transforms.Normalize(mean=[0.507, 0.487, 0.441],
                                     std=[0.267, 0.256, 0.276])

In [5]:
dataset = datasets.ImageFolder(
        train_path_path,
        transforms.Compose([
        transforms.RandomRotation(degrees=45),
            
        transforms.RandomResizedCrop(70),
        transforms.ColorJitter(.3,.3,.3,.3),
        transforms.RandomHorizontalFlip(),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))
for _ in range (19):
    aug_data = datasets.ImageFolder(
        train_path_path,
        transforms.Compose([
        transforms.RandomRotation(degrees=45),
            
        transforms.RandomResizedCrop(70),
        transforms.ColorJitter(.3,.3,.3,.3),
        transforms.RandomHorizontalFlip(),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))
    dataset=torch.utils.data.ConcatDataset([dataset,aug_data])
valset = datasets.ImageFolder(
        test_path_path,
        transforms.Compose([
        transforms.RandomRotation(degrees=45),
            
        transforms.RandomResizedCrop(70),
        transforms.ColorJitter(.3,.3,.3,.3),
        transforms.RandomHorizontalFlip(),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))
for _ in range (19):
    aug_data = datasets.ImageFolder(
        test_path_path,
        transforms.Compose([
        transforms.RandomRotation(degrees=45),
            
        transforms.RandomResizedCrop(70),
        transforms.ColorJitter(.3,.3,.3,.3),
        transforms.RandomHorizontalFlip(),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))
    valset=torch.utils.data.ConcatDataset([valset,aug_data])

train_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=True,
        num_workers=8, pin_memory=True,drop_last=True)

test_loader = torch.utils.data.DataLoader(
        valset, batch_size=batch_size, shuffle=True,
        num_workers=8, pin_memory=True,drop_last=True)

In [6]:
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

In [7]:
class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [8]:
class ResNet(nn.Module):

    def __init__(
        self,
        block: Type[Union[Bottleneck]],
        layers: List[int],
        num_classes: int = 10,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 20, layers[0])
        self.layer2 = self._make_layer(block, 40, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 80, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(80 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]

    def _make_layer(self, block: Type[Union[Bottleneck]], planes: int, blocks: int,
                    stride: int = 1, dilate: bool = False) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = torch.flatten(x,1)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


def _resnet(
    arch: str,
    block: Type[Union[Bottleneck]],
    layers: List[int],
    pretrained: bool,
    progress: bool,
    **kwargs: Any
) -> ResNet:
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

def _resnext(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch], progress=progress)
        model.load_state_dict(state_dict)
    return model

In [9]:
def Model(pretrained: bool = False, progress: bool = True, **kwargs):
    
    kwargs['groups'] = 4
    kwargs['width_per_group'] = 32
    return _resnext('resnext', Bottleneck, [3, 8, 8], pretrained, progress, **kwargs)


In [10]:
model = Model()
model = model.cuda()
criterion = nn.CrossEntropyLoss().cuda()

In [11]:
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print (name, param.data)

In [12]:
from torchsummary import summary
summary(model,(3,96,96))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 48, 48]           9,408
       BatchNorm2d-2           [-1, 64, 48, 48]             128
              ReLU-3           [-1, 64, 48, 48]               0
         MaxPool2d-4           [-1, 64, 24, 24]               0
            Conv2d-5           [-1, 40, 24, 24]           2,560
       BatchNorm2d-6           [-1, 40, 24, 24]              80
              ReLU-7           [-1, 40, 24, 24]               0
            Conv2d-8           [-1, 40, 24, 24]           3,600
       BatchNorm2d-9           [-1, 40, 24, 24]              80
             ReLU-10           [-1, 40, 24, 24]               0
           Conv2d-11           [-1, 80, 24, 24]           3,200
      BatchNorm2d-12           [-1, 80, 24, 24]             160
           Conv2d-13           [-1, 80, 24, 24]           5,120
      BatchNorm2d-14           [-1, 80,

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100)

In [14]:
saving_path="/home/jupyter-deeplearning/res_model"
trn_loss_list = []
val_loss_list = []
val_acc_list = []
total_epoch=500
model_char="2.0"
model_name=""
patience=10
start_early_stop_check=0
saving_start_epoch=10

for epoch in range(total_epoch):
    trn_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        if torch.cuda.is_available():
            inputs=inputs.cuda()
            labels=labels.cuda()
        # grad init
        optimizer.zero_grad()
        # forward propagation
        output= model(inputs)
        # calculate loss
        loss=criterion(output, labels)
        # back propagation 
        loss.backward()
        # weight update
        optimizer.step()
        
        # trn_loss summary
        trn_loss += loss.item()
        # del (memory issue)
        del loss
        del output
    with torch.no_grad():
        val_loss = 0.0
        cor_match = 0
        for j, val in enumerate(test_loader):
            val_x, val_label = val
            if torch.cuda.is_available():
                val_x = val_x.cuda()
                val_label =val_label.cuda()
            val_output = model(val_x)
            v_loss = criterion(val_output, val_label)
            val_loss += v_loss
            _, predicted=torch.max(val_output,1)
            cor_match+=np.count_nonzero(predicted.cpu().detach()==val_label.cpu().detach())
    del val_output
    del v_loss
    del predicted
    
    scheduler.step()
    
    
    
    trn_loss_list.append(trn_loss/len(train_loader))
    val_loss_list.append(val_loss/len(test_loader))
    val_acc=cor_match/(len(test_loader)*batch_size)
    val_acc_list.append(val_acc)
    now = time.localtime()
    print ("%04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec))

    print("epoch: {}/{} | trn loss: {:.4f} | val loss: {:.4f} | val accuracy: {:.4f}% \n".format(
                epoch+1, total_epoch, trn_loss / len(train_loader), val_loss / len(test_loader), val_acc*100
            ))
    
    
    if epoch+1>2:
        if val_loss_list[-1]>val_loss_list[-2]:
            start_early_stop_check=1
    else:
        val_loss_min=val_loss_list[-1]
        
    if start_early_stop_check:
        early_stop_temp=val_loss_list[-patience:]
        if all(early_stop_temp[i]<early_stop_temp[i+1] for i in range (len(early_stop_temp)-1)):
            print("Early stop!")
            break
            
    if epoch+1>saving_start_epoch:
        if val_loss_list[-1]<val_loss_min:
            if os.path.isfile(model_name):
                os.remove(model_name)
            val_loss_min=val_loss_list[-1]
            model_name=saving_path+"Custom_model_"+model_char+"_{:.3f}".format(val_loss_min)
            torch.save(model, model_name)
            print("Model replaced and saved as ",model_name)

2020/11/23 17:13:02
epoch: 1/500 | trn loss: 2.0324 | val loss: 1.8622 | val accuracy: 30.5739% 

2020/11/23 17:13:34
epoch: 2/500 | trn loss: 1.7865 | val loss: 1.7368 | val accuracy: 35.6170% 

2020/11/23 17:14:06
epoch: 3/500 | trn loss: 1.6579 | val loss: 1.6480 | val accuracy: 39.5182% 

2020/11/23 17:14:38
epoch: 4/500 | trn loss: 1.5483 | val loss: 1.5507 | val accuracy: 43.6999% 

2020/11/23 17:15:11
epoch: 5/500 | trn loss: 1.4450 | val loss: 1.4642 | val accuracy: 46.9201% 

2020/11/23 17:15:43
epoch: 6/500 | trn loss: 1.3643 | val loss: 1.4520 | val accuracy: 47.9718% 

2020/11/23 17:16:15
epoch: 7/500 | trn loss: 1.2813 | val loss: 1.3744 | val accuracy: 50.3456% 

2020/11/23 17:16:47
epoch: 8/500 | trn loss: 1.2005 | val loss: 1.3456 | val accuracy: 52.3488% 

2020/11/23 17:17:20
epoch: 9/500 | trn loss: 1.1311 | val loss: 1.3007 | val accuracy: 53.5707% 

2020/11/23 17:17:53
epoch: 10/500 | trn loss: 1.0671 | val loss: 1.2779 | val accuracy: 55.0731% 

2020/11/23 17:18:25

2020/11/23 17:55:58
epoch: 80/500 | trn loss: 0.2760 | val loss: 1.3233 | val accuracy: 65.4247% 

2020/11/23 17:56:31
epoch: 81/500 | trn loss: 0.2743 | val loss: 1.3165 | val accuracy: 65.1292% 

2020/11/23 17:57:03
epoch: 82/500 | trn loss: 0.2709 | val loss: 1.2870 | val accuracy: 65.5799% 

2020/11/23 17:57:36
epoch: 83/500 | trn loss: 0.2675 | val loss: 1.3380 | val accuracy: 65.5599% 

2020/11/23 17:58:09
epoch: 84/500 | trn loss: 0.2677 | val loss: 1.3657 | val accuracy: 64.5583% 

2020/11/23 17:58:42
epoch: 85/500 | trn loss: 0.2707 | val loss: 1.3404 | val accuracy: 64.8988% 

2020/11/23 17:59:14
epoch: 86/500 | trn loss: 0.2650 | val loss: 1.3823 | val accuracy: 64.7436% 

2020/11/23 17:59:47
epoch: 87/500 | trn loss: 0.2660 | val loss: 1.3398 | val accuracy: 65.2344% 

2020/11/23 18:00:20
epoch: 88/500 | trn loss: 0.2670 | val loss: 1.3652 | val accuracy: 64.7436% 

2020/11/23 18:00:53
epoch: 89/500 | trn loss: 0.2643 | val loss: 1.3775 | val accuracy: 64.6234% 

2020/11/23

2020/11/23 18:41:24
epoch: 163/500 | trn loss: 0.0901 | val loss: 1.4898 | val accuracy: 67.8285% 

2020/11/23 18:41:57
epoch: 164/500 | trn loss: 0.0878 | val loss: 1.4820 | val accuracy: 68.1991% 

2020/11/23 18:42:30
epoch: 165/500 | trn loss: 0.0894 | val loss: 1.4815 | val accuracy: 68.2742% 

2020/11/23 18:43:03
epoch: 166/500 | trn loss: 0.0867 | val loss: 1.5072 | val accuracy: 67.7484% 

2020/11/23 18:43:36
epoch: 167/500 | trn loss: 0.0883 | val loss: 1.4676 | val accuracy: 68.1040% 

2020/11/23 18:44:09
epoch: 168/500 | trn loss: 0.0904 | val loss: 1.4712 | val accuracy: 68.3494% 

2020/11/23 18:44:41
epoch: 169/500 | trn loss: 0.0875 | val loss: 1.4414 | val accuracy: 68.9904% 

2020/11/23 18:45:14
epoch: 170/500 | trn loss: 0.0865 | val loss: 1.4616 | val accuracy: 68.1440% 

2020/11/23 18:45:47
epoch: 171/500 | trn loss: 0.0881 | val loss: 1.4320 | val accuracy: 68.6649% 

2020/11/23 18:46:20
epoch: 172/500 | trn loss: 0.0863 | val loss: 1.4946 | val accuracy: 67.3628% 



2020/11/23 19:26:30
epoch: 245/500 | trn loss: 0.0640 | val loss: 1.5026 | val accuracy: 68.7500% 

2020/11/23 19:27:05
epoch: 246/500 | trn loss: 0.0615 | val loss: 1.4991 | val accuracy: 68.9253% 

2020/11/23 19:27:40
epoch: 247/500 | trn loss: 0.0605 | val loss: 1.5067 | val accuracy: 68.7300% 

2020/11/23 19:28:15
epoch: 248/500 | trn loss: 0.0620 | val loss: 1.5123 | val accuracy: 68.7800% 

2020/11/23 19:28:50
epoch: 249/500 | trn loss: 0.0617 | val loss: 1.5021 | val accuracy: 68.7800% 

2020/11/23 19:29:25
epoch: 250/500 | trn loss: 0.0626 | val loss: 1.4925 | val accuracy: 68.9203% 

2020/11/23 19:30:00
epoch: 251/500 | trn loss: 0.0610 | val loss: 1.4961 | val accuracy: 69.1356% 

2020/11/23 19:30:35
epoch: 252/500 | trn loss: 0.0631 | val loss: 1.4895 | val accuracy: 69.0855% 

2020/11/23 19:31:09
epoch: 253/500 | trn loss: 0.0618 | val loss: 1.4897 | val accuracy: 68.7800% 

2020/11/23 19:31:44
epoch: 254/500 | trn loss: 0.0584 | val loss: 1.5109 | val accuracy: 68.7200% 



2020/11/23 20:17:48
epoch: 327/500 | trn loss: 0.0582 | val loss: 1.5175 | val accuracy: 68.4395% 

2020/11/23 20:18:27
epoch: 328/500 | trn loss: 0.0578 | val loss: 1.5144 | val accuracy: 68.5296% 

2020/11/23 20:19:07
epoch: 329/500 | trn loss: 0.0558 | val loss: 1.5140 | val accuracy: 68.6498% 

2020/11/23 20:19:48
epoch: 330/500 | trn loss: 0.0538 | val loss: 1.5230 | val accuracy: 68.7250% 

2020/11/23 20:20:28
epoch: 331/500 | trn loss: 0.0552 | val loss: 1.5151 | val accuracy: 68.6949% 

2020/11/23 20:21:06
epoch: 332/500 | trn loss: 0.0579 | val loss: 1.5208 | val accuracy: 68.3393% 

2020/11/23 20:21:45
epoch: 333/500 | trn loss: 0.0598 | val loss: 1.5305 | val accuracy: 68.4195% 

2020/11/23 20:22:23
epoch: 334/500 | trn loss: 0.0569 | val loss: 1.5010 | val accuracy: 68.5697% 

2020/11/23 20:23:04
epoch: 335/500 | trn loss: 0.0580 | val loss: 1.5164 | val accuracy: 68.4545% 

2020/11/23 20:23:43
epoch: 336/500 | trn loss: 0.0550 | val loss: 1.4824 | val accuracy: 68.9653% 



2020/11/23 21:11:34
epoch: 409/500 | trn loss: 0.0550 | val loss: 1.5134 | val accuracy: 68.4495% 

2020/11/23 21:12:12
epoch: 410/500 | trn loss: 0.0554 | val loss: 1.5075 | val accuracy: 68.9253% 

2020/11/23 21:12:51
epoch: 411/500 | trn loss: 0.0576 | val loss: 1.5094 | val accuracy: 68.7851% 

2020/11/23 21:13:31
epoch: 412/500 | trn loss: 0.0564 | val loss: 1.5097 | val accuracy: 68.4545% 

2020/11/23 21:14:08
epoch: 413/500 | trn loss: 0.0560 | val loss: 1.5159 | val accuracy: 68.5447% 

2020/11/23 21:14:46
epoch: 414/500 | trn loss: 0.0556 | val loss: 1.5429 | val accuracy: 67.9337% 

2020/11/23 21:15:24
epoch: 415/500 | trn loss: 0.0556 | val loss: 1.4896 | val accuracy: 68.7400% 

2020/11/23 21:16:06
epoch: 416/500 | trn loss: 0.0553 | val loss: 1.5168 | val accuracy: 68.5096% 

2020/11/23 21:16:45
epoch: 417/500 | trn loss: 0.0549 | val loss: 1.5005 | val accuracy: 69.1607% 

2020/11/23 21:17:23
epoch: 418/500 | trn loss: 0.0559 | val loss: 1.5166 | val accuracy: 69.0505% 



2020/11/23 22:04:53
epoch: 491/500 | trn loss: 0.0560 | val loss: 1.5018 | val accuracy: 68.7250% 

2020/11/23 22:05:32
epoch: 492/500 | trn loss: 0.0567 | val loss: 1.4999 | val accuracy: 69.0254% 

2020/11/23 22:06:10
epoch: 493/500 | trn loss: 0.0576 | val loss: 1.5214 | val accuracy: 68.6799% 

2020/11/23 22:06:50
epoch: 494/500 | trn loss: 0.0586 | val loss: 1.4954 | val accuracy: 68.8902% 

2020/11/23 22:07:28
epoch: 495/500 | trn loss: 0.0558 | val loss: 1.5443 | val accuracy: 68.4095% 

2020/11/23 22:08:06
epoch: 496/500 | trn loss: 0.0541 | val loss: 1.5204 | val accuracy: 68.5847% 

2020/11/23 22:08:46
epoch: 497/500 | trn loss: 0.0556 | val loss: 1.5142 | val accuracy: 68.7450% 

2020/11/23 22:09:24
epoch: 498/500 | trn loss: 0.0568 | val loss: 1.5201 | val accuracy: 68.6649% 

2020/11/23 22:10:03
epoch: 499/500 | trn loss: 0.0560 | val loss: 1.5346 | val accuracy: 68.1440% 

2020/11/23 22:10:42
epoch: 500/500 | trn loss: 0.0539 | val loss: 1.5004 | val accuracy: 68.9203% 



In [15]:
dataset = datasets.ImageFolder(
        train_path_path,
        transforms.Compose([
        transforms.RandomRotation(degrees=45),
        transforms.Resize(120),    
        transforms.ColorJitter(.3,.3,.3,.3),
        transforms.RandomHorizontalFlip(),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))
for _ in range (19):
    aug_data = datasets.ImageFolder(
        train_path_path,
        transforms.Compose([
        transforms.RandomRotation(degrees=45),
        transforms.Resize(120),    
        transforms.ColorJitter(.3,.3,.3,.3),
        transforms.RandomHorizontalFlip(),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))
    dataset=torch.utils.data.ConcatDataset([dataset,aug_data])
valset = datasets.ImageFolder(
        test_path_path,
        transforms.Compose([
        transforms.RandomRotation(degrees=45),
        transforms.Resize(120),
        transforms.ColorJitter(.3,.3,.3,.3),
        transforms.RandomHorizontalFlip(),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))
for _ in range (19):
    aug_data = datasets.ImageFolder(
        test_path_path,
        transforms.Compose([
        transforms.RandomRotation(degrees=45),
        transforms.Resize(120),    
        transforms.ColorJitter(.3,.3,.3,.3),
        transforms.RandomHorizontalFlip(),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))
    valset=torch.utils.data.ConcatDataset([valset,aug_data])

train_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=True,
        num_workers=8, pin_memory=True,drop_last=True)

test_loader = torch.utils.data.DataLoader(
        valset, batch_size=batch_size, shuffle=True,
        num_workers=8, pin_memory=True,drop_last=True)

In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50)

In [ ]:
#fine tuning

total_epoch=200
model_char="2.0"
model_name=""
patience=10
start_early_stop_check=0
saving_start_epoch=10

for epoch in range(total_epoch):
    trn_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        if torch.cuda.is_available():
            inputs=inputs.cuda()
            labels=labels.cuda()
        # grad init
        optimizer.zero_grad()
        # forward propagation
        output= model(inputs)
        # calculate loss
        loss=criterion(output, labels)
        # back propagation 
        loss.backward()
        # weight update
        optimizer.step()
        
        # trn_loss summary
        trn_loss += loss.item()
        # del (memory issue)
        del loss
        del output
    with torch.no_grad():
        val_loss = 0.0
        cor_match = 0
        for j, val in enumerate(test_loader):
            val_x, val_label = val
            if torch.cuda.is_available():
                val_x = val_x.cuda()
                val_label =val_label.cuda()
            val_output = model(val_x)
            v_loss = criterion(val_output, val_label)
            val_loss += v_loss
            _, predicted=torch.max(val_output,1)
            cor_match+=np.count_nonzero(predicted.cpu().detach()==val_label.cpu().detach())
    del val_output
    del v_loss
    del predicted
    
    scheduler.step()
    
    
    
    trn_loss_list.append(trn_loss/len(train_loader))
    val_loss_list.append(val_loss/len(test_loader))
    val_acc=cor_match/(len(test_loader)*batch_size)
    val_acc_list.append(val_acc)
    now = time.localtime()
    print ("%04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec))

    print("epoch: {}/{} | trn loss: {:.4f} | val loss: {:.4f} | val accuracy: {:.4f}% \n".format(
                epoch+1, total_epoch, trn_loss / len(train_loader), val_loss / len(test_loader), val_acc*100
            ))
    
    
    if epoch+1>2:
        if val_loss_list[-1]>val_loss_list[-2]:
            start_early_stop_check=1
    else:
        val_loss_min=val_loss_list[-1]
        
    if start_early_stop_check:
        early_stop_temp=val_loss_list[-patience:]
        if all(early_stop_temp[i]<early_stop_temp[i+1] for i in range (len(early_stop_temp)-1)):
            print("Early stop!")
            break
            
    if epoch+1>saving_start_epoch:
        if val_loss_list[-1]<val_loss_min:
            if os.path.isfile(model_name):
                os.remove(model_name)
            val_loss_min=val_loss_list[-1]
            model_name="Custom_model_"+model_char+"_{:.3f}".format(val_loss_min)
            torch.save(model, model_name)
            print("Model replaced and saved as ",model_name)

2020/11/23 22:12:04
epoch: 1/200 | trn loss: 0.0340 | val loss: 1.1267 | val accuracy: 73.8181% 

2020/11/23 22:13:24
epoch: 2/200 | trn loss: 0.0337 | val loss: 1.0665 | val accuracy: 75.4708% 

2020/11/23 22:14:45
epoch: 3/200 | trn loss: 0.0286 | val loss: 1.1431 | val accuracy: 73.9483% 

2020/11/23 22:16:05
epoch: 4/200 | trn loss: 0.0303 | val loss: 1.1103 | val accuracy: 75.6510% 

2020/11/23 22:17:26
epoch: 5/200 | trn loss: 0.0278 | val loss: 1.1243 | val accuracy: 75.3155% 

2020/11/23 22:18:47
epoch: 6/200 | trn loss: 0.0283 | val loss: 1.2901 | val accuracy: 72.6663% 

2020/11/23 22:20:08
epoch: 7/200 | trn loss: 0.0282 | val loss: 1.1475 | val accuracy: 74.8648% 

2020/11/23 22:21:29
epoch: 8/200 | trn loss: 0.0286 | val loss: 1.1833 | val accuracy: 73.5527% 

2020/11/23 22:22:49
epoch: 9/200 | trn loss: 0.0266 | val loss: 1.2012 | val accuracy: 74.1486% 

2020/11/23 22:24:10
epoch: 10/200 | trn loss: 0.0178 | val loss: 1.1789 | val accuracy: 74.8397% 

2020/11/23 22:25:31

2020/11/23 23:59:25
epoch: 83/200 | trn loss: 0.0011 | val loss: 1.0960 | val accuracy: 76.9481% 

2020/11/24 00:00:43
epoch: 84/200 | trn loss: 0.0007 | val loss: 1.0856 | val accuracy: 76.8930% 

2020/11/24 00:02:00
epoch: 85/200 | trn loss: 0.0012 | val loss: 1.1168 | val accuracy: 75.9465% 

2020/11/24 00:03:18
epoch: 86/200 | trn loss: 0.0015 | val loss: 1.1192 | val accuracy: 76.4774% 

2020/11/24 00:04:36
epoch: 87/200 | trn loss: 0.0007 | val loss: 1.0941 | val accuracy: 77.2786% 

2020/11/24 00:05:54
epoch: 88/200 | trn loss: 0.0006 | val loss: 1.1048 | val accuracy: 77.3488% 

2020/11/24 00:07:11
epoch: 89/200 | trn loss: 0.0025 | val loss: 1.1449 | val accuracy: 75.9115% 

2020/11/24 00:08:29
epoch: 90/200 | trn loss: 0.0009 | val loss: 1.1143 | val accuracy: 76.8229% 

2020/11/24 00:09:47
epoch: 91/200 | trn loss: 0.0006 | val loss: 1.1272 | val accuracy: 76.4123% 

2020/11/24 00:11:04
epoch: 92/200 | trn loss: 0.0005 | val loss: 1.0905 | val accuracy: 77.6342% 

2020/11/24

2020/11/24 01:46:34
epoch: 166/200 | trn loss: 0.0003 | val loss: 1.0936 | val accuracy: 77.6042% 

2020/11/24 01:47:51
epoch: 167/200 | trn loss: 0.0003 | val loss: 1.0901 | val accuracy: 77.5491% 

2020/11/24 01:49:08
epoch: 168/200 | trn loss: 0.0003 | val loss: 1.0976 | val accuracy: 77.4940% 

2020/11/24 01:50:26
epoch: 169/200 | trn loss: 0.0003 | val loss: 1.0884 | val accuracy: 77.6893% 

2020/11/24 01:51:43
epoch: 170/200 | trn loss: 0.0003 | val loss: 1.0924 | val accuracy: 77.6292% 

2020/11/24 01:53:00
epoch: 171/200 | trn loss: 0.0003 | val loss: 1.0695 | val accuracy: 77.7494% 

2020/11/24 01:54:18
epoch: 172/200 | trn loss: 0.0003 | val loss: 1.1011 | val accuracy: 77.2686% 

2020/11/24 01:55:36
epoch: 173/200 | trn loss: 0.0003 | val loss: 1.0854 | val accuracy: 77.6142% 

2020/11/24 01:56:54
epoch: 174/200 | trn loss: 0.0003 | val loss: 1.0928 | val accuracy: 77.4790% 

2020/11/24 01:58:13
epoch: 175/200 | trn loss: 0.0003 | val loss: 1.1020 | val accuracy: 77.2686% 



In [ ]:
plt.figure(figsize=(15,15))
plt.ylabel("val_accuracy")
plt.xlabel("epoch")
plt.plot(val_acc_list)
plt.grid()

In [ ]:
# val_acc_list=np.array(val_acc_list)
# np.savetxt("ver_2.0.txt", val_acc_list, delimiter=',')

### Evaluation

In [ ]:
# import torch
# import pandas as pd
# import argparse
# import time
# import torchvision
# import torchvision.transforms as transforms
# from torch.utils.data import Dataset, DataLoader

In [ ]:
# model.eval()

In [ ]:
# normalize = transforms.Normalize(mean=[0.507, 0.487, 0.441],
#                                      std=[0.267, 0.256, 0.276])
# test_transform = transforms.Compose([
#         transforms.ToTensor(),
#         normalize
#     ])

In [ ]:
# test_dataset = torchvision.datasets.ImageFolder('./data/test', transform=test_transform)

In [ ]:
# test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=4, shuffle=False)

In [ ]:
# Category = []
# for input, _ in test_loader:
#     input = input.cuda()
#     output = model(input)
#     output = torch.argmax(output, dim=1)
#     Category = Category + output.tolist()

In [ ]:
# Id = list(range(0, 8000))
# samples = {
#    'Id': Id,
#    'Category': Category 
# }
# df = pd.DataFrame(samples, columns=['Id', 'Category'])

# df.to_csv('submission_2.0_2.csv', index=False)
# print('Done!!')